# Practice Project: The System Bootloader 💻

## 🎓 Module 01 Summary
Before we start coding, let's recap the critical shifts from C to C++:

| Concept | C Approach | Modern C++ Approach |
| :--- | :--- | :--- |
| Compiler | gcc | g++ (links Standard Library) |
| Extensions | .h / .c | .hpp / .cpp |
| Build System | Make (Manual Flags) | CMake (Targets & Modules) |
| Dependencies | apt-get / Copy-paste | FetchContent / vcpkg |
| Printing | printf | std::cout |

---

## 🚀 The Mission: System Bootloader

You are writing the startup software for a server (or a spaceship, if you prefer). 
Your program needs to read a configuration file (config.json) and print the system status.

Since writing a JSON parser from scratch in C is painful (string parsing hell), we will use the industry-standard C++ library: nlohmann/json.

### 📋 Requirements

1.  Project Structure:
    * src/main.cpp
    * src/config_loader.cpp
    * include/config_loader.hpp
    * CMakeLists.txt
    * config.json
2.  Library: Use FetchContent in CMake to download nlohmann/json.
3.  Functionality:
    * Read system_name (string).
    * Read cpu_count (int).
    * Read enabled_modules (array of strings).
    * Print them using std::cout.

---

## 🛠️ Step 1: Create the Folder Structure

Create a new folder named practice_bootloader inside this module.
Inside it, create src, include, and a blank CMakeLists.txt.

## 🛠️ Step 2: The CMake Recipe

Copy this into your CMakeLists.txt. This is your first time using FetchContent!

```cmake
cmake_minimum_required(VERSION 3.14)
project(Bootloader)

set(CMAKE_CXX_STANDARD 17)

# --- Dependency Management ---
include(FetchContent)

# Download the JSON library release archive (official recommendation)
set(JSON_BuildTests OFF CACHE INTERNAL "") # we don't need the library's tests
FetchContent_Declare(
    json
    URL https://github.com/nlohmann/json/releases/download/v3.12.0/json.tar.xz
)
FetchContent_MakeAvailable(json)

# --- Project Setup ---
include_directories(include)

add_executable(boot_system 
    src/main.cpp 
    src/config_loader.cpp
)

# Our headers live in include/
target_include_directories(boot_system PRIVATE ${CMAKE_CURRENT_SOURCE_DIR}/include)

# Link the JSON interface target; it supplies include paths and C++ features
target_link_libraries(boot_system PRIVATE nlohmann_json::nlohmann_json)
```

## 🛠️ Step 3: The C++ Code

Now, write the code. Here are some hints (spoilers below if you get stuck).

### config.json
```json
{
    "system_name": "Orion-Server-01",
    "cpu_count": 16,
    "modules": ["Auth", "Database", "Cache"]
}
```

### include/config_loader.hpp
Define a namespace Bootloader. Declare a function load_config(std::string path)`

### src/config_loader.cpp
Implementation.
You will need:

```cpp
#include <fstream> // File Stream (like FILE*)
#include <nlohmann/json.hpp>
using json = nlohmann::json;
```

Hint: The library allows json data = json::parse(file_stream);

### src/main.cpp
Call the function.

---

## 🔎 Solution Reference

Try to implement it yourself first! If you get stuck on the syntax, check the blocks below.

In [1]:
// --- include/config_loader.hpp ---
#pragma once
#include <string>

namespace Bootloader {
    void load_config(const std::string& path);
}

In [ ]:
// --- src/config_loader.cpp ---
#include "config_loader.hpp"
#include <iostream>
#include <fstream>
#include <nlohmann/json.hpp>

// Alias for cleaner code
using json = nlohmann::json;

namespace Bootloader {
    void load_config(const std::string& path) {
        std::ifstream file(path);
        if (!file.is_open()) {
            std::cerr << "Error: Could not open config file!" << std::endl;
            return;
        }

        // Parse JSON directly from file stream
        json data = json::parse(file);

        // Accessing data (Notice how it looks like Python dictionaries)
        std::string name = data["system_name"];
        int cpu = data["cpu_count"];

        std::cout << "Booting System: " << name << std::endl;
        std::cout << "CPUs Detected:  " << cpu << std::endl;
        std::cout << "Modules: " << std::endl;

        // Range-based for loop (Modern C++)
        for (const auto& mod : data["modules"]) {
            std::cout << "  [+] " << mod << std::endl;
        }
    }
}

## 🧪 Run It

In your terminal:

```bash
cd practice01
mkdir build
cd build
cmake ..
make

# Ensure config.json is in the folder you run the app from!
# If config.json is in the project root, you might need to copy it or run from root.
cp ../config.json .
./boot_system
```